In [1]:
from pathlib import Path
import os

# Ici on remonte d'un dossier (ETL -> racine projet)
project_root = Path.cwd().parent
os.chdir(project_root)

# Vérifions que ça marche bien
print("Racine projet :", project_root)


Racine projet : c:\Users\antoi\Documents\Work_Learn\Rugby


In [5]:
import pandas as pd

df = pd.read_excel(r"./data/raw/STATS INDIVIDUELLES.xlsx", sheet_name="Promedio partidos", header=None)
# display(df.head())


In [14]:
import pandas as pd

df = pd.read_excel(r"./data/raw/STATS INDIVIDUELLES.xlsx", sheet_name="AGEN I", header=None)
print(df.iloc[1][~df.iloc[1].isna()])


1                Duel
6               Passe
11           Plaquage
16               Ruck
21                JAP
26      Réception JAP
31          En-avants
32           Penalité
33    Ballon récupéré
34       Nº d’actions
35       Moyenne T.I.
36     Niveau T.I (%)
37             Valeur
Name: 1, dtype: object


In [15]:
import pandas as pd
import re
import unicodedata
from fuzzywuzzy import process

def normaliser_nom(nom):
    if not isinstance(nom, str):
        return ""
    nom = unicodedata.normalize('NFKD', nom).encode('ASCII', 'ignore').decode('utf-8')
    nom = re.sub(r"[.\-]", " ", nom)
    nom = re.sub(r"\s+", " ", nom).strip()
    return nom.upper()

def generer_cle_abregee(prenom, nom_famille):
    initiales = "".join([p[0].upper() for p in prenom.strip().split() if p])
    nom = nom_famille.upper()
    return normaliser_nom(f"{initiales} {nom}")

def extraire_noms_complets(path_xlsx, feuille="Promedio partidos"):
    df = pd.read_excel(path_xlsx, sheet_name=feuille, header=None)
    noms = df.iloc[2:, 1].dropna().astype(str)
    correspondance = {}
    for nom_complet in noms:
        try:
            nom = nom_complet.strip()
            nom_parts = nom.split(",")
            if len(nom_parts) == 2:
                nom_famille = nom_parts[0].strip().title()
                prenom = nom_parts[1].strip().title()
                cle = generer_cle_abregee(prenom, nom_famille)
                correspondance[cle] = (prenom, nom_famille)
        except Exception:
            continue
    return correspondance

def trouver_prenom_nom(abbr, noms_complets):
    cle = normaliser_nom(abbr)
    best_match, score = process.extractOne(cle, noms_complets.keys())
    if score > 80:
        return pd.Series(noms_complets[best_match])
    return pd.Series((None, None))

def normaliser_nom_colonne(nom):
    if not isinstance(nom, str):
        return ""
    nom = unicodedata.normalize('NFKD', nom).encode('ASCII', 'ignore').decode('utf-8')
    nom = re.sub(r"[.\-]", " ", nom)
    nom = re.sub(r"\s+", " ", nom).strip()
    return nom.upper()


c:\Users\antoi\Documents\Work_Learn\Rugby\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [18]:
def transformer_fichier_excel(path_xlsx, feuille_a_exclure="Promedio partidos", sortie_csv="./data/transformed/Rugby_Stats.csv"):
    xls = pd.ExcelFile(path_xlsx)
    feuilles = [sheet for sheet in xls.sheet_names if feuille_a_exclure.lower() not in sheet.lower()]
    noms_complets = extraire_noms_complets(path_xlsx)
    toutes_donnees = []

    for feuille in feuilles:
        print(f"📄 Traitement de la feuille : {feuille}")
        df_raw = pd.read_excel(xls, sheet_name=feuille, header=None)
        actions = df_raw.iloc[1].fillna(method="ffill")
        niveaux = df_raw.iloc[3]
        columns = []

        for i in range(len(actions)):
            if i == 0:
                columns.append("Joueuse")
            else:
                action = str(actions[i]).strip()
                niveau = str(niveaux[i]).strip()

                if niveau == "M" or niveau.lower() == "nan":
                    # Garder uniquement certaines actions sans niveau
                    actions_sans_niveau_autorisees = ["EN-AVANTS", "PENALITE", "BALLON RECUPERE"]
                    nom_action = normaliser_nom_colonne(action)
                    if nom_action in actions_sans_niveau_autorisees:
                        columns.append(nom_action)
                    else:
                        columns.append(None)
                    continue

                try:
                    niveau_int = int(float(niveau))
                    if action.lower() != "nan":
                        columns.append(normaliser_nom_colonne(f"{action}_{niveau_int}"))
                    else:
                        columns.append(None)
                except ValueError:
                    columns.append(None)


        df_data = df_raw.iloc[4:].copy()
        df_data.columns = columns
        df_data = df_data.loc[:, df_data.columns.notna()]
        df_data = df_data[df_data["Joueuse"].notna()]
        df_data[["Numero", "Joueuse"]] = df_data["Joueuse"].astype(str).str.extract(r"(\d+)\s*-\s*(.*)")
        df_data = df_data[df_data["Joueuse"].notna()]
        df_data = df_data[~df_data["Joueuse"].str.strip().str.upper().eq("GENERAL")]

        # Transforme en format long
        df_long = df_data.melt(id_vars=["Numero", "Joueuse"],
                               var_name="Action_Niveau",
                               value_name="Nb_actions")

        # Supprime les colonnes _M (déjà exclues en amont, mais par sécurité)
        df_long = df_long[~df_long["Action_Niveau"].str.contains("_M$", regex=True, na=False)]

        # Séparation Action et Niveau (Niveau peut être NaN ici)
        df_long["Action"] = df_long["Action_Niveau"].str.extract(r"^(.+?)(?:_\d+)?$")
        df_long["Niveau"] = df_long["Action_Niveau"].str.extract(r"_(\d+)")
        df_long["Niveau"] = pd.to_numeric(df_long["Niveau"], errors="coerce")

        df_long["Match"] = feuille
        df_long = df_long.dropna(subset=["Nb_actions"])

        df_long["Joueuse"] = df_long["Joueuse"].astype(str).str.strip()
        df_long[["Prenom", "Nom"]] = df_long["Joueuse"].apply(trouver_prenom_nom, noms_complets=noms_complets)

        df_long["Nb_actions"] = pd.to_numeric(df_long["Nb_actions"], errors="coerce")
        df_long = df_long.dropna(subset=["Nb_actions"])

        toutes_donnees.append(df_long)

    df_final = pd.concat(toutes_donnees, ignore_index=True)
    non_trouvees = df_final[df_final["Prenom"].isna()]["Joueuse"].unique()

    if len(non_trouvees):
        print("\n❗️Joueuses non reconnues (à vérifier manuellement) :")
        for j in non_trouvees:
            print(f" - {j}")

    df_final = df_final[["Numero", "Prenom", "Nom", "Match", "Action", "Niveau", "Nb_actions"]]
    df_final.to_csv(sortie_csv, index=False)
    print(f"✅ Export terminé : {sortie_csv}")
transformer_fichier_excel(r"./data/raw/STATS INDIVIDUELLES.xlsx")


📄 Traitement de la feuille : PAU I
📄 Traitement de la feuille : COLOMIERS I
📄 Traitement de la feuille : BLAGNAC I
📄 Traitement de la feuille : AGEN I
📄 Traitement de la feuille : CASTRES I
📄 Traitement de la feuille : BAYONNE I
📄 Traitement de la feuille : AURILLAC I
📄 Traitement de la feuille : PAU II
📄 Traitement de la feuille : COLOMIERS II
📄 Traitement de la feuille : BLAGNAC II
📄 Traitement de la feuille : AGEN II
📄 Traitement de la feuille : CASTRES II
📄 Traitement de la feuille : BAYONNE II
📄 Traitement de la feuille : AURILLAC II
📄 Traitement de la feuille : TOULON
📄 Traitement de la feuille : BRIVE
✅ Export terminé : ./data/transformed/Rugby_Stats.csv


In [ ]:

# def transformer_fichier_excel(path_xlsx, feuille_a_exclure="Promedio partidos", sortie_csv="./data/transformed/Rugby_Stats.csv"):
#     xls = pd.ExcelFile(path_xlsx)
#     feuilles = [sheet for sheet in xls.sheet_names if feuille_a_exclure.lower() not in sheet.lower()]
#     noms_complets = extraire_noms_complets(path_xlsx)
#     toutes_donnees = []

#     for feuille in feuilles:
#         print(f"📄 Traitement de la feuille : {feuille}")
#         df_raw = pd.read_excel(xls, sheet_name=feuille, header=None)
#         actions = df_raw.iloc[1].fillna(method="ffill")
#         niveaux = df_raw.iloc[3]
#         columns = []

#         for i in range(len(actions)):
#             if i == 0:
#                 columns.append("Joueuse")
#             else:
#                 action = str(actions[i]).strip()
#                 niveau = str(niveaux[i]).strip()
#                 if niveau == "M" or niveau.lower() == "nan":
#                     columns.append(None)
#                     continue
#                 try:
#                     niveau_int = int(float(niveau))
#                     if action != 'nan':
#                         columns.append(normaliser_nom_colonne(f"{action}_{niveau_int}"))
#                     else:
#                         columns.append(None)
#                 except ValueError:
#                     columns.append(None)

#         df_data = df_raw.iloc[4:].copy()
#         df_data.columns = columns
#         df_data = df_data.loc[:, df_data.columns.notna()]
#         df_data = df_data[df_data["Joueuse"].notna()]
#         df_data[["Numero", "Joueuse"]] = df_data["Joueuse"].astype(str).str.extract(r"(\d+)\s*-\s*(.*)")
#         df_data = df_data[df_data["Joueuse"].notna()]
#         df_data = df_data[~df_data["Joueuse"].str.strip().str.upper().eq("GENERAL")]
#         df_long = df_data.melt(id_vars=["Numero", "Joueuse"],
#                                var_name="Action_Niveau",
#                                value_name="Nb_actions")
#         df_long = df_long[~df_long["Action_Niveau"].str.contains("_M$", regex=True)]
#         df_long["Action"] = df_long["Action_Niveau"].str.extract(r"(.+)_\d+")
#         df_long["Niveau"] = df_long["Action_Niveau"].str.extract(r"_(\d+)").astype(int)
#         df_long["Match"] = feuille
#         df_long = df_long.dropna(subset=["Nb_actions"])
#         df_long["Joueuse"] = df_long["Joueuse"].astype(str).str.strip()
#         df_long[["Prenom", "Nom"]] = df_long["Joueuse"].apply(trouver_prenom_nom, noms_complets=noms_complets)
#         df_long["Nb_actions"] = pd.to_numeric(df_long["Nb_actions"], errors="coerce")
#         df_long = df_long.dropna(subset=["Nb_actions"])
#         toutes_donnees.append(df_long)

#     df_final = pd.concat(toutes_donnees, ignore_index=True)
#     non_trouvees = df_final[df_final["Prenom"].isna()]["Joueuse"].unique()

#     if len(non_trouvees):
#         print("\n❗️Joueuses non reconnues (à vérifier manuellement) :")
#         for j in non_trouvees:
#             print(f" - {j}")

#     df_final = df_final[["Numero", "Prenom", "Nom", "Match", "Action", "Niveau", "Nb_actions"]]
#     df_final.to_csv(sortie_csv, index=False)
#     print(f"✅ Export terminé : {sortie_csv}")


# # Exemple d'utilisation
# transformer_fichier_excel(r"./data/raw/STATS INDIVIDUELLES.xlsx")


In [19]:
import pandas as pd

# Lire le fichier CSV
df = pd.read_csv("./data/transformed/Rugby_Stats.csv")

# Afficher le type de chaque valeur dans les colonnes pour les trois premières lignes
for index, row in df.head(3).iterrows():
    print(f"Ligne {index}:")
    for column in df.columns:
        print(f"    {column}: {row[column]} (Type: {type(row[column])})")
    print()


Ligne 0:
    Numero: 1 (Type: <class 'int'>)
    Prenom: Cléa (Type: <class 'str'>)
    Nom: Vauloup (Type: <class 'str'>)
    Match: PAU I (Type: <class 'str'>)
    Action: DUEL (Type: <class 'str'>)
    Niveau: 0.0 (Type: <class 'float'>)
    Nb_actions: 2.0 (Type: <class 'float'>)

Ligne 1:
    Numero: 2 (Type: <class 'int'>)
    Prenom: Arwen (Type: <class 'str'>)
    Nom: Lavuia (Type: <class 'str'>)
    Match: PAU I (Type: <class 'str'>)
    Action: DUEL (Type: <class 'str'>)
    Niveau: 0.0 (Type: <class 'float'>)
    Nb_actions: 2.0 (Type: <class 'float'>)

Ligne 2:
    Numero: 3 (Type: <class 'int'>)
    Prenom: Manon (Type: <class 'str'>)
    Nom: Marchesin (Type: <class 'str'>)
    Match: PAU I (Type: <class 'str'>)
    Action: DUEL (Type: <class 'str'>)
    Niveau: 0.0 (Type: <class 'float'>)
    Nb_actions: 1.0 (Type: <class 'float'>)



In [ ]:
# import sqlite3
# import pandas as pd

# # Lire le fichier CSV
# df = pd.read_csv("./data/transformed/Rugby_Stats.csv")

# # Connexion SQLite
# conn = sqlite3.connect("./data/transformed/Rugby_Stats.db")
# cursor = conn.cursor()

# # Créer les tables
# cursor.execute('''
# CREATE TABLE IF NOT EXISTS Joueuse (
#     id_joueuse INTEGER PRIMARY KEY AUTOINCREMENT,
#     prenom TEXT NOT NULL,
#     nom TEXT NOT NULL,
#     UNIQUE(prenom, nom)
# )
# ''')

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS Match (
#     id_match INTEGER PRIMARY KEY AUTOINCREMENT,
#     nom_match TEXT NOT NULL UNIQUE
# )
# ''')

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS Action (
#     id_action INTEGER PRIMARY KEY AUTOINCREMENT,
#     nom_action TEXT NOT NULL UNIQUE
# )
# ''')

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS Niveau (
#     id_niveau INTEGER PRIMARY KEY,
#     description TEXT NOT NULL
# )
# ''')

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS Statistiques (
#     id_stat INTEGER PRIMARY KEY AUTOINCREMENT,
#     id_joueuse INTEGER,
#     id_match INTEGER,
#     id_action INTEGER,
#     id_niveau INTEGER,
#     numero INTEGER,
#     nb_actions INTEGER NOT NULL,
#     FOREIGN KEY (id_joueuse) REFERENCES Joueuse(id_joueuse),
#     FOREIGN KEY (id_match) REFERENCES Match(id_match),
#     FOREIGN KEY (id_action) REFERENCES Action(id_action),
#     FOREIGN KEY (id_niveau) REFERENCES Niveau(id_niveau)
# )
# ''')

# # Insertion
# for index, row in df.iterrows():
#     prenom = row['Prenom']
#     nom = row['Nom']
#     numero = int(row['Numero']) if not pd.isna(row['Numero']) else None

#     # Joueuse
#     cursor.execute('''
#     INSERT OR IGNORE INTO Joueuse (prenom, nom) VALUES (?, ?)
#     ''', (prenom, nom))
#     cursor.execute('SELECT id_joueuse FROM Joueuse WHERE prenom = ? AND nom = ?', (prenom, nom))
#     id_joueuse = cursor.fetchone()[0]

#     # Match
#     cursor.execute('INSERT OR IGNORE INTO Match (nom_match) VALUES (?)', (row['Match'],))
#     cursor.execute('SELECT id_match FROM Match WHERE nom_match = ?', (row['Match'],))
#     id_match = cursor.fetchone()[0]

#     # Action
#     cursor.execute('INSERT OR IGNORE INTO Action (nom_action) VALUES (?)', (row['Action'],))
#     cursor.execute('SELECT id_action FROM Action WHERE nom_action = ?', (row['Action'],))
#     id_action = cursor.fetchone()[0]

#     # Niveau
#     niveau_id = int(row['Niveau']) if not pd.isna(row['Niveau']) else 0
#     description = f"{row['Action']}_{niveau_id}"
#     cursor.execute('INSERT OR IGNORE INTO Niveau (id_niveau, description) VALUES (?, ?)', (niveau_id, description))

#     # Statistique
#     nb = int(row['Nb_actions']) if not pd.isna(row['Nb_actions']) else 0
#     cursor.execute('''
#     INSERT INTO Statistiques (id_joueuse, id_match, id_action, id_niveau, numero, nb_actions)
#     VALUES (?, ?, ?, ?, ?, ?)
#     ''', (id_joueuse, id_match, id_action, niveau_id, numero, nb))

# # Commit & close
# conn.commit()
# conn.close()


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("./data/transformed/Rugby_Stats.db")


query = '''
SELECT j.prenom, j.nom, SUM(s.nb_actions) AS total_actions
FROM Statistiques s
JOIN Joueuse j ON s.id_joueuse = j.id_joueuse
GROUP BY j.id_joueuse
ORDER BY total_actions DESC
'''
df = pd.read_sql_query(query, conn)
print(df)


           prenom          nom  total_actions
0           Alice       Henaux            557
1          Alexia     Cazalbou            460
2          Maelys      Onillon            425
3           Adèle     Lachurie            423
4         Heloïse      Seletti            356
5            Cléa      Vauloup            322
6            Moea       Lickel            321
7           Arwen       Lavuia            321
8            Jade       Guindé            310
9             Lou       Cambus            298
10           Lise        Llach            285
11        Alessia        Coste            256
12        Justine       Pradel            252
13          Alice       Bouvet            249
14           Elsa     Moutinho            247
15         Jeanne  Beheregaray            238
16          Manon    Marchesin            232
17         Sofaia         Fono            229
18           Mado       Bidard            229
19         Agathe       Lacaze            220
20            Zoe      Toustou    

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("./data/transformed/Rugby_Stats.db")

query = '''
SELECT j.prenom, j.nom, m.nom_match, s.numero, a.nom_action, n.id_niveau, s.nb_actions
FROM Statistiques s
JOIN Joueuse j ON s.id_joueuse = j.id_joueuse
JOIN Match m ON s.id_match = m.id_match
JOIN Action a ON s.id_action = a.id_action
JOIN Niveau n ON s.id_niveau = n.id_niveau
WHERE j.nom = 'Onillon'
'''
df = pd.read_sql_query(query, conn)
print(df)



     prenom      nom  nom_match  numero nom_action  id_niveau  nb_actions
0    Maelys  Onillon  BLAGNAC I      21       DUEL          1           1
1    Maelys  Onillon  BLAGNAC I      21       DUEL          2           3
2    Maelys  Onillon  BLAGNAC I      21      PASSE          0           1
3    Maelys  Onillon  BLAGNAC I      21      PASSE          2           2
4    Maelys  Onillon  BLAGNAC I      21      PASSE          3           2
..      ...      ...        ...     ...        ...        ...         ...
126  Maelys  Onillon      BRIVE      12   PLAQUAGE          0           2
127  Maelys  Onillon      BRIVE      12   PLAQUAGE          1           5
128  Maelys  Onillon      BRIVE      12   PLAQUAGE          2           6
129  Maelys  Onillon      BRIVE      12   PLAQUAGE          3           2
130  Maelys  Onillon      BRIVE      12       RUCK          3           7

[131 rows x 7 columns]
